In [3]:
#!pip install pandas-datareader

In [80]:
import numpy as np
import pandas as pd
import pandas_datareader
import datetime as dt
import os

PATH = os.getcwd()

In [5]:
#Grab sentiments
#Big-cap
big_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USbig_Sent12_21.csv'))
big_sent_all['date'] = pd.to_datetime(big_sent_all['date']).dt.date
#Mid-cap
mid_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USmed_Sent12_21.csv'))
mid_sent_all['date'] = pd.to_datetime(mid_sent_all['date']).dt.date
#Small-caps
small_sent_all = pd.read_csv(os.path.join(PATH, 'dataSent12_21Good', 'USsmall_Sent12_21.csv'))
small_sent_all['date'] = pd.to_datetime(small_sent_all['date']).dt.date

## Sentiment Indicators

In [6]:
## The sentiment dataset in the 'dataSent12_21Good' folder is in long format, with each column being a different sentiment indicator.
# We create a table for each sentiment indicator, and pivot them to into familiar wide format:
# Each table will align all stocks by date and those which has no data for given date will be fill with NaN

#Make a table for each sentiment indicator
big_sent_tables = {}
big_sent_nan_tables = {}
for i in big_sent_all.columns[2:]:   
    big_sent_pivot = big_sent_all.pivot(index="date", columns="stock", values= i)
    big_sent_pivot.index = pd.to_datetime(big_sent_pivot.index)
    big_sent_tables['big_'+i] = big_sent_pivot
    big_sent_pivot.to_csv(os.path.join(PATH,'Tables','big_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder

print('List of tables created: ',big_sent_tables.keys())
print('RCV Table:')
big_sent_tables['big_RCV'].head()

List of tables created:  dict_keys(['big_RCV', 'big_RVT', 'big_positivePartscr', 'big_negativePartscr', 'big_splogscr', 'big_linscr'])
RCV Table:


stock,AAL,AAPL,ABBV,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
2012-01-03,NaN,0.000,NaN,0.000,0.000,NaN,NaN,NaN,NaN,NaN,...,2.632,NaN,NaN,0.000,NaN,0.000,0.000,0.000,NaN,33.333
2012-01-04,0.000,41.667,NaN,44.444,-14.286,0.000,0.000,0.000,0.000,0.000,...,47.692,0.000,0.000,30.000,0.000,22.222,0.000,37.500,0.0,43.478
2012-01-05,25.000,45.455,NaN,0.000,33.333,38.889,13.333,-7.692,4.167,-9.091,...,38.571,0.000,25.000,57.333,38.462,49.383,51.852,48.148,-20.0,38.889
2012-01-06,46.666,45.395,NaN,-33.333,-73.333,57.384,-60.000,-43.750,41.935,-35.714,...,26.250,33.335,-57.142,27.941,-28.571,14.706,36.765,54.412,NaN,20.000


In [7]:
#Same procedure for mid and small-cap

#Mid companies
mid_sent_tables = {}
mid_sent_nan_tables = {}
for i in mid_sent_all.columns[2:]:   
    mid_sent_pivot = mid_sent_all.pivot(index="date", columns="stock", values= i)
    mid_sent_pivot.index = pd.to_datetime(mid_sent_pivot.index)
    mid_sent_tables['mid_'+i] = mid_sent_pivot
    mid_sent_pivot.to_csv(os.path.join(PATH,'Tables','mid_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder
     
#Small companies
small_sent_tables = {}
small_sent_nan_tables = {}
for i in small_sent_all.columns[2:]:   
    small_sent_pivot = small_sent_all.pivot(index="date", columns="stock", values= i)
    small_sent_pivot.index = pd.to_datetime(small_sent_pivot.index)
    small_sent_tables['small_'+i] = small_sent_pivot
    small_sent_pivot.to_csv(os.path.join(PATH,'Tables','small_{}.csv'.format(i))) #Store in csv format in the 'Tables' folder

## Stock Price Data

In [102]:
big_tickers = big_sent_tables['big_RCV'].columns
mid_tickers = mid_sent_tables['mid_RCV'].columns
small_tickers = small_sent_tables['small_RCV'].columns

In [155]:
# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
big_tickers = big_sent_tables['big_RCV'].columns.drop('ABBV') # ABBV returns weekly returns starting on Tuesday in  for some unknown reason

# We would like all available data from 2012/01/02 until 2021/12/01.
start_date = big_sent_tables['big_RCV'].index[0].strftime('%Y-%m-%d') 
end_date = big_sent_tables['big_RCV'].index[-1].strftime('%Y-%m-%d')

# User pandas_reader to load the desired data in a weekly format. As simple as that.
big_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(big_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
big_panel_data = pd.DataFrame(big_panel_data)
big_panel_data.to_csv(os.path.join(PATH,'Tables','big_prices.csv')) #Store in csv format in the 'Tables' folder
big_panel_data.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,5.279352,12.917507,32.785946,21.673082,37.985802,19.400175,5.43,182.610001,41.543427,5.321594,...,18.209999,45.038452,54.356091,20.957052,23.435894,23.872055,21.554796,46.660233,12.920536,55.891045
2012-01-09,5.628165,12.838299,32.114758,21.506247,38.034267,20.562208,5.66,178.419998,42.985672,5.691869,...,18.430000,44.970192,54.859123,21.923719,23.442875,24.554789,22.053816,47.087288,13.735708,55.733471
2012-01-16,6.005264,12.853288,34.060356,21.820961,39.307842,21.139105,6.42,190.929993,43.227558,6.087976,...,19.330000,44.603268,55.791176,21.704714,23.405643,24.586340,22.746492,48.249840,14.027818,57.447239
2012-01-23,7.711626,13.678369,33.117287,21.531370,38.304211,20.809454,6.82,195.369995,43.063442,6.277416,...,23.090000,43.536598,56.249821,21.040129,23.515018,23.475946,22.046371,48.012577,13.905539,56.357250
2012-01-30,9.088028,14.057577,33.329693,21.523542,38.421871,22.391792,7.08,187.679993,45.136681,6.751020,...,24.969999,43.784065,56.738026,22.052107,24.906601,23.873417,22.813522,49.056492,14.469369,55.759735


In [87]:
#Same procedure for mid and small
#Mid-caps
mid_tickers = mid_sent_tables['mid_RCV'].columns
start_date = mid_sent_tables['mid_RCV'].index[0].strftime('%Y-%m-%d')
end_date = mid_sent_tables['mid_RCV'].index[-1].strftime('%Y-%m-%d')
mid_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(mid_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
mid_panel_data = pd.DataFrame(mid_panel_data)
mid_panel_data.to_csv(os.path.join(PATH,'Tables','mid_prices.csv')) #Store in csv format in the 'Tables' folder

C:\Users\ericb\anaconda3\lib\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'INTL', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


In [ ]:
#Small-caps
small_tickers = small_sent_tables['small_RCV'].columns
start_date = small_sent_tables['small_RCV'].index[0].strftime('%Y-%m-%d')
end_date = small_sent_tables['small_RCV'].index[-1].strftime('%Y-%m-%d')
small_panel_data = pandas_datareader.yahoo.daily.YahooDailyReader(small_tickers, interval='w', start=start_date, end=end_date).read()['Adj Close'] 
small_panel_data = pd.DataFrame(small_panel_data)
small_panel_data.to_csv(os.path.join(PATH,'Tables','small_prices.csv')) #Store in csv format in the 'Tables' folder

In [156]:
big_panel_data.describe()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
count,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,...,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000,518.000000
mean,29.853003,46.208520,76.403275,55.778054,100.567580,43.311936,23.769633,1228.443396,86.017446,20.245517,...,54.004903,172.671980,98.232929,38.866198,106.222788,40.722022,39.716230,82.847009,24.377418,59.714999
std,13.165709,37.902158,22.562542,29.474035,48.648670,9.642800,31.566222,1038.071789,30.405967,9.447480,...,20.889458,108.886416,35.902708,9.485844,63.517024,8.993890,8.890773,29.136350,4.951926,8.584846
min,5.279352,12.119164,30.683222,21.506247,35.505150,18.120352,1.670000,178.419998,41.543427,5.321594,...,17.780001,43.536598,52.966915,20.957052,23.405643,23.475946,20.970749,46.494160,12.920536,28.654728
25%,17.079329,21.081754,64.160900,33.962798,59.090282,38.240262,3.650000,332.880005,65.314680,13.154113,...,37.352500,72.228413,78.439589,32.618833,51.127288,34.041476,32.530491,61.721056,21.400329,57.597895
50%,31.701316,29.332281,79.777470,41.319750,87.409885,45.946999,9.840000,809.265015,78.896275,19.514085,...,53.095001,147.995499,90.945660,36.268644,79.447773,38.961329,42.324793,68.720882,24.132570,61.392267
75%,40.459659,51.459453,88.960617,75.740543,138.991318,50.422192,29.515001,1816.377502,100.601908,26.946706,...,69.429998,246.156433,104.375635,46.205420,159.927593,50.045856,46.344869,102.208635,27.132939,65.256172
max,56.988731,164.560349,126.690041,127.927086,235.679306,59.332623,155.410004,3719.340088,186.083435,47.319759,...,96.699997,457.618835,212.720993,61.726746,247.840775,57.480377,58.148563,150.261490,37.384274,72.813080


## Log Retruns

In [159]:
#Log return:
big_log_ret = np.log(big_panel_data) - np.log(big_panel_data.shift(1))
big_log_ret.to_csv(os.path.join(PATH,'Tables','big_log_ret.csv')) #Store in csv format in the 'Tables' folder

#Percentage change:
#big_pct_change = big_panel_data.pct_change()
#big_pct_changet.to_csv(os.path.join(PATH,'Tables','big_pct_change.csv'))

big_log_ret.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,0.063980,-0.006151,-0.020684,-0.007728,0.001275,0.058173,0.041485,-0.023212,0.034128,0.067266,...,0.012009,-0.001517,0.009212,0.045094,0.000298,0.028198,0.022887,0.009111,0.061181,-0.002823
2012-01-16,0.064853,0.001167,0.058818,0.014528,0.032937,0.027670,0.125994,0.067767,0.005611,0.067277,...,0.047679,-0.008193,0.016847,-0.010040,-0.001589,0.001284,0.030925,0.024389,0.021043,0.030286
2012-01-23,0.250093,0.062216,-0.028079,-0.013360,-0.025864,-0.015717,0.060441,0.022988,-0.003804,0.030643,...,0.177741,-0.024205,0.008187,-0.031098,0.004662,-0.046215,-0.031263,-0.004930,-0.008755,-0.019156
2012-01-30,0.164229,0.027346,0.006393,-0.000364,0.003067,0.073287,0.037414,-0.040157,0.047021,0.072735,...,0.078275,0.005668,0.008642,0.046977,0.057494,0.016789,0.034205,0.021510,0.039747,-0.010659


In [90]:
#Same for mid and small-caps
#Log return:
mid_log_ret = np.log(mid_panel_data) - np.log(mid_panel_data.shift(1))
mid_log_ret.to_csv(os.path.join(PATH,'Tables','mid_log_ret.csv')) #Store in csv format in the 'Tables' folder
small_log_ret = np.log(small_panel_data) - np.log(small_panel_data.shift(1))
small_log_ret.to_csv(os.path.join(PATH,'Tables','small_log_ret.csv')) #Store in csv format in the 'Tables' folder

#Percentage change:
#mid_pct_change = mid_panel_data.pct_change()
#mid_pct_changet.to_csv(os.path.join(PATH,'Tables','mid_pct_change.csv'))
#small_pct_change = small_panel_data.pct_change()
#small_pct_changet.to_csv(os.path.join(PATH,'Tables','small_pct_change.csv'))

## Ranking Tables

In [165]:
big_rankings = big_log_ret.rank(1, ascending=False, method='first')
big_rankings.to_csv(os.path.join(PATH,'Tables','big_rankings.csv')) #Store in csv format in the 'Tables' folder

#Same for Mid and Small-caps
mid_rankings = mid_log_ret.rank(1, ascending=False, method='first')
mid_rankings.to_csv(os.path.join(PATH,'Tables','mid_rankings.csv')) #Store in csv format in the 'Tables' folder
small_rankings = small_log_ret.rank(1, ascending=False, method='first')
small_rankings.to_csv(os.path.join(PATH,'Tables','small_rankings.csv')) #Store in csv format in the 'Tables' folder
big_rankings.head()

Symbols,AAL,AAPL,ABC,ABT,ADP,AIG,AMD,AMZN,AXP,BAC,...,UAL,UNH,UPS,USB,V,VZ,WFC,WMT,WY,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-09,10.0,88.0,95.0,89.0,78.0,13.0,28.0,98.0,33.0,9.0,...,63.0,82.0,65.0,21.0,79.0,36.0,45.0,66.0,12.0,86.0
2012-01-16,11.0,76.0,15.0,60.0,31.0,41.0,2.0,8.0,71.0,9.0,...,21.0,90.0,55.0,93.0,82.0,75.0,34.0,46.0,53.0,38.0
2012-01-23,1.0,9.0,92.0,73.0,90.0,75.0,10.0,25.0,55.0,16.0,...,3.0,88.0,42.0,95.0,45.0,102.0,96.0,57.0,66.0,79.0
2012-01-30,1.0,53.0,82.0,90.0,86.0,12.0,38.0,104.0,28.0,13.0,...,10.0,84.0,79.0,29.0,19.0,67.0,40.0,61.0,35.0,96.0
